In [3]:
import random

import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.nn import functional as F

MAX_LENGTH = 1024
INPUT_SOS = '<|context|>'
INPUT_EOS = '<|endofcontext|>'
OUTPUT_SOS = '<|belief|>'
OUTPUT_EOS = '<|endofbelief|>'
UNK_TOKEN = '<unk>'

In [4]:
# picking the most free GPU resource as cuda device
import subprocess
import sys
import os


if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO


def get_free_gpu():
    gpu_stats = subprocess.check_output(
        ["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"]
    )
    gpu_df = pd.read_csv(
        StringIO(gpu_stats.decode("utf-8")),
        names=["memory.used", "memory.free"],
        skiprows=1,
    )
    print("GPU usage:\n{}".format(gpu_df))
    gpu_df["memory.free"] = gpu_df["memory.free"].map(
        lambda x: int(x.rstrip(" MiB"))
    )
    idx = gpu_df["memory.free"].idxmax()
    print(
        "Returning GPU{} with {} free MiB".format(
            idx, gpu_df.iloc[idx]["memory.free"]
        )
    )
    return idx


cmd = "export CUDA_VISIBLE_DEVICES=1,2,3,4,5,6"
os.popen(cmd)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using {device}")
if torch.cuda.is_available():
    free_gpu_id = get_free_gpu()
    print(f"using GPU id: {free_gpu_id}")
    torch.cuda.set_device(free_gpu_id)


using cuda
GPU usage:
  memory.used memory.free
0    4733 MiB   19535 MiB
1    4733 MiB   19535 MiB
2    4733 MiB   19535 MiB
3   20026 MiB    4242 MiB
4   15209 MiB    9059 MiB
5    4033 MiB   20235 MiB
Returning GPU5 with 20235 free MiB
using GPU id: 5


In [5]:
with open('../resources/gpt2/train.history_belief') as fp:
    raw_train_text = fp.read()

In [6]:
raw_train_text = raw_train_text.replace('<|endofcontext|>', ' <|endofcontext|>')
raw_train_text = raw_train_text.replace('<|endoftext|>', '')

In [7]:
train_text = raw_train_text.split('\n')

In [8]:
input_text = []
beliefs = []

for text in train_text:
    split_index = text.find('<|belief|>')  
    input_text.append(text[:split_index-1].split())
    beliefs.append(text[split_index:].split())

In [9]:
train_data = pd.DataFrame(data={'input_text': input_text, 'beliefs': beliefs})

In [10]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [11]:
input_lang = Lang('input_lang')
output_lang = Lang('output_lang')
for word in raw_train_text.split():
    input_lang.addWord(word)
    output_lang.addWord(word)
input_lang.addWord(UNK_TOKEN)
output_lang.addWord(UNK_TOKEN)
input_lang.EOS = INPUT_EOS
output_lang.EOS = OUTPUT_EOS

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [13]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [15]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index.get(word, lang.word2index[UNK_TOKEN]) for word in sentence]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(lang.word2index[lang.EOS])
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [38]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[output_lang.word2index[OUTPUT_SOS]]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == output_lang.word2index[OUTPUT_EOS]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [17]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [18]:
train_tuples = [t[1:] for t in train_data.itertuples()]

In [39]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_tuples))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [20]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [46]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_lang.word2index[OUTPUT_SOS]]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == output_lang.word2index[OUTPUT_EOS]:
                decoded_words.append(OUTPUT_EOS)
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]
    
def belief_to_state_list(belief):
    belief_list = [token for token in belief if token not in [OUTPUT_SOS, OUTPUT_EOS]]
    belief_list = [slot.split() for slot in ' '.join(belief_list).split(',')]
    return belief_list
def belief_to_state_dict(belief):
    belief_list = belief_to_state_list(belief)
    state_dict = {}
    for state in belief_list:
        if len(state) < 3: continue
        domain = state[0]
        slot = state[1]
        sub_slot = None
        rest = state[2:]
        if slot == 'book':
            sub_slot = state[2]
            rest = state[3:]
        value = ' '.join(rest)
        d = state_dict.get(domain, {})
        if sub_slot:
            ss = d.get(slot, {})
            ss.update({
                sub_slot: value
            })
            d.update({slot: ss})
        else:
            d.update({slot: value})
        state_dict.update({domain: d})
    return state_dict


def match_slot(true, pred):
    pred_state = belief_to_state_dict(pred)
    true_list = belief_to_state_list(true)
    slot_matches = []
    for i, state in enumerate(true_list):
        if len(state) < 3: continue
        slot_matches.append(False)
        domain = state[0]
        if domain not in pred_state.keys(): continue
        
        slot = state[1]
        if slot not in pred_state[domain].keys(): continue
        
        if slot != 'book':
            true_value = " ".join(state[2:])
            pred_value = pred_state[domain][slot]
        else:
            sub_slot = state[2]
            if sub_slot not in pred_state[domain][slot]: continue
            true_value = " ".join(state[3:])
            pred_value = pred_state[domain][slot][sub_slot]
        
        if true_value != pred_value: continue
        slot_matches[i] = True
            
    all_match = sum(slot_matches) == len(true_list)
    
    return all_match, slot_matches

In [22]:
with open('../resources/gpt2/val.history_belief') as fp:
    raw_val_text = fp.read()

In [23]:
raw_val_text = raw_val_text.replace('<|endofcontext|>', ' <|endofcontext|>')
raw_val_text = raw_val_text.replace('<|endoftext|>', '')

In [24]:
val_text = raw_val_text.split('\n')

In [25]:
input_text = []
beliefs = []

for text in val_text:
    split_index = text.find('<|belief|>')  
    input_text.append(text[:split_index-1].split())
    beliefs.append(text[split_index:].split())

In [26]:
val_data = pd.DataFrame(data={'input_text': input_text, 'beliefs': beliefs})

In [27]:
val_tuples = [t[1:] for t in val_data.itertuples()]

In [40]:
from tqdm import tqdm
def evaluateRandomly(encoder, decoder, n=10):
    match_results = []
    for i in range(n):
        pair = random.choice(val_tuples)
        # print('>', pair[0])
        # print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        # output_sentence = ' '.join(output_words)
        # print('<', output_words)
        match_results.append(match_slot(pair[1], output_words))
        # print('matching: ', match_slot(pair[1], output_words))
        # print('')
    print(get_accuracy(match_results))
    
def evaluate_valset(encoder, decoder):
    match_results = []
    for pair in tqdm(val_tuples):
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        match_results.append(match_slot(pair[1], output_words))
    
    print(get_accuracy(match_results))
        

In [41]:
def get_accuracy(results):
    total_states = len(results)
    total_slots = sum([len(result[1]) for result in results])
    total_correct_states = sum([result[0] for result in results])
    total_correct_slots = sum([sum(result[1]) for result in results])
    return {
        'joint_accuracy': total_correct_states / total_states,
        'slot_accuracy': total_correct_slots / total_slots
    }

In [43]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 20000, print_every=100)

0m 24s (- 81m 54s) (100 0%) 4.8502
0m 45s (- 74m 47s) (200 1%) 3.2326
1m 8s (- 74m 35s) (300 1%) 3.0032
1m 28s (- 72m 27s) (400 2%) 2.8887
1m 50s (- 72m 2s) (500 2%) 2.7363
2m 12s (- 71m 22s) (600 3%) 2.4808
2m 35s (- 71m 31s) (700 3%) 2.3524
2m 56s (- 70m 25s) (800 4%) 2.3188
3m 16s (- 69m 36s) (900 4%) 2.2795
3m 38s (- 69m 4s) (1000 5%) 2.1999
3m 58s (- 68m 9s) (1100 5%) 2.3343
4m 19s (- 67m 41s) (1200 6%) 2.0694
4m 40s (- 67m 14s) (1300 6%) 2.1116
5m 2s (- 67m 1s) (1400 7%) 2.2105
5m 24s (- 66m 42s) (1500 7%) 2.2464
5m 45s (- 66m 10s) (1600 8%) 2.2196
6m 6s (- 65m 40s) (1700 8%) 2.0445
6m 26s (- 65m 6s) (1800 9%) 2.0447
6m 46s (- 64m 30s) (1900 9%) 2.2229
7m 7s (- 64m 8s) (2000 10%) 2.0060
7m 28s (- 63m 44s) (2100 10%) 2.0950
7m 51s (- 63m 32s) (2200 11%) 2.1252
8m 12s (- 63m 8s) (2300 11%) 1.8681
8m 33s (- 62m 42s) (2400 12%) 2.0450
8m 55s (- 62m 26s) (2500 12%) 1.8364
9m 17s (- 62m 7s) (2600 13%) 1.5975
9m 41s (- 62m 7s) (2700 13%) 1.6459
10m 1s (- 61m 36s) (2800 14%) 1.5792
10m 2

In [44]:
evaluateRandomly(encoder1, attn_decoder1, 2000)

{'joint_accuracy': 0.0075, 'slot_accuracy': 0.1947565543071161}


In [47]:
evaluate_valset(encoder1, attn_decoder1)

100%|██████████| 7375/7375 [06:52<00:00, 17.88it/s]

{'joint_accuracy': 0.008949152542372881, 'slot_accuracy': 0.19583711521006722}


In [212]:
belief_to_state_dict(train_data['beliefs'][4])

{'hotel': {'name': 'not mentioned',
  'area': 'not mentioned',
  'parking': 'yes',
  'pricerange': 'cheap',
  'stars': 'not mentioned',
  'internet': 'not mentioned',
  'type': 'hotel',
  'book': {'stay': '2', 'day': 'tuesday', 'people': '6'}}}

In [224]:
val_belief_sample = ['<|belief|>', 'attraction', 'type', 'college', ',', 'attraction', 'name', 'christ', 'college', ',', 'attraction', 'area', 'not', 'mentioned', '<|endofbelief|>']

In [225]:
belief_to_state_dict(val_belief_sample)

{'attraction': {'type': 'college',
  'name': 'christ college',
  'area': 'not mentioned'}}

In [226]:
val_belief_true_sample = ['<|belief|>', 'attraction', 'type', 'college', ',', 'attraction', 'name', 'christ', 'college', ',', 'attraction', 'area', 'not', 'mentioned', '<|endofbelief|>']

In [240]:
match_slot(val_belief_true_sample, val_belief_sample)

(True, [True, True, True])